In [ ]:
from qdrant_client.models import Distance, VectorParams
from qdrant_client import QdrantClient

In [ ]:
client = QdrantClient(url="http://localhost:6333")
client.create_collection(
    collection_name="test_collection",
    vectors_config=VectorParams(size=4, distance=Distance.DOT),
)